In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from pprint import pprint

from joblib import Parallel, delayed
import joblib

In [2]:
#movieUser_df = pd.read_csv('IMDbMovieUserPivotTrimmed.csv')
refined_dataset = pd.read_csv('IMDbRefined2 old.csv')

knn_model = joblib.load('IMDbModel.h5')
#movieUser_df = pd.read_csv('movieUser.csv')

In [3]:
refined_dataset = refined_dataset.drop('Unnamed: 0', axis=1)

In [37]:
newTitleID = "tt0120815"    
newUserID = "645ad00ea917e05432316a5c"
newPrimaryTitle = "Saving Private Ryan"
newRating = "10.0"
row = [newTitleID, newUserID, newPrimaryTitle, newRating]
row

['tt0120815', '645ad00ea917e05432316a5c', 'Saving Private Ryan', '10.0']

In [38]:
refined_dataset.loc[len(refined_dataset)] = row

In [39]:
# newTitleID = "tt0110357"    
# newUserID = "645b6f50b130d14687d91f12"
# newPrimaryTitle = "The Lion King"
# newRating = "10.0"
# row = [newTitleID, newUserID, newPrimaryTitle, newRating]
# row

In [40]:
# refined_dataset.loc[len(refined_dataset)] = row

In [41]:
refined_dataset

,titleID,userID,primaryTitle,rating
0,tt0034583,ur0011104,Casablanca,1.0
1,tt0034583,ur0011762,Casablanca,10.0
2,tt0034583,ur0012295,Casablanca,7.0
3,tt0034583,ur0013691,Casablanca,10.0
4,tt0034583,ur0016344,Casablanca,10.0
...,...,...,...,...
103777,tt8946378,ur99907711,Knives Out,5.0
103778,tt8946378,641c87d06a97e629837fc079,Knives Out,8.0
103779,tt0110357,645ad00ea917e05432316a5c,The Lion King,7.0
103780,tt0120586,645ad00ea917e05432316a5c,American History X,10.0


In [42]:
movieUser_df = refined_dataset.pivot_table(
    index='userID',
     columns='titleID',
      ## Replacing all movies users haven't rated with a rating of 0
      values='rating').fillna(0)

In [43]:
#movieUser_df = movieUser_df.reset_index(); 

In [44]:
#movieUser_df.columns

In [45]:
#movieUser_df.columns

In [46]:
movie_list = movieUser_df.columns
movieUser_scipy_df = csr_matrix(movieUser_df.values)

In [47]:
knn_model.fit(movieUser_scipy_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [48]:
simUsers = []
userDistances = []
highestMovies = []
recommendedMovies = []

In [49]:
def similar_users(user, n = 5):
    knn_input = np.asarray([movieUser_df.values[user]])
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
        
    for i in range(1,len(distances[0])):
        simUsers.append(indices[0][i]+1)
        userDistances.append(distances[0][i])
    return indices.flatten()[1:] + 1, distances.flatten()[1:]

In [50]:
def recommend_movies(n):
    n = min(len(mean_ratings_list),n)
    recommendedMovies.append(list(movie_list[np.argsort(mean_ratings_list)[::-1][:n]]))
    return recommendedMovies

In [51]:
target_user_id = "645ad00ea917e05432316a5c"
target_user = np.where(movieUser_df.index == target_user_id)[0][0]
no_of_highest = 15
no_of_similar_users = 5
no_of_movies = 12

In [52]:
highestMovies.append(list(refined_dataset[refined_dataset['userID'] == target_user_id].sort_values('rating', ascending=False)['titleID'])[:no_of_highest])

In [53]:
similar_user_list, distance_list = similar_users(target_user, no_of_similar_users)
weighted_list = distance_list/np.sum(distance_list)
similar_user_ratings = movieUser_df.values[similar_user_list]
weighted_list = weighted_list[:,np.newaxis] + np.zeros(len(movie_list))
ratings_matrix = weighted_list*similar_user_ratings
mean_ratings_list = ratings_matrix.sum(axis =0)

In [54]:
recommend_movies(no_of_movies)

[['tt0361748',
  'tt1136608',
  'tt0095016',
  'tt0114746',
  'tt0948470',
  'tt0078748',
  'tt0383574',
  'tt0167260',
  'tt0910970',
  'tt0119217',
  'tt0119654',
  'tt0118715']]

In [55]:
dict1 = {'Users_Top_Movies': highestMovies, 'Similar_Users': simUsers, 'Sim_User_distances': userDistances, 'Recommendatons': recommendedMovies}

In [56]:
dict1

{'Users_Top_Movies': [['tt0110357', 'tt0120586', 'tt0120815']],
 'Similar_Users': [500, 351, 5772, 3043, 1211],
 'Sim_User_distances': [0.2738890210558872,
  0.3019007026627678,
  0.3662757494755221,
  0.3662757494755221,
  0.3662757494755221],
 'Recommendatons': [['tt0361748',
   'tt1136608',
   'tt0095016',
   'tt0114746',
   'tt0948470',
   'tt0078748',
   'tt0383574',
   'tt0167260',
   'tt0910970',
   'tt0119217',
   'tt0119654',
   'tt0118715']]}

In [24]:
movieUser_df.iloc[3135]

titleID
tt0034583    0.0
tt0050083    0.0
tt0054215    0.0
tt0060196    0.0
tt0062622    0.0
            ... 
tt3896198    0.0
tt7131622    0.0
tt7286456    0.0
tt8579674    0.0
tt8946378    0.0
Name: ur104874236, Length: 200, dtype: float64

In [25]:
recommendedMovies

[['tt8946378',
  'tt0120586',
  'tt0133093',
  'tt0126029',
  'tt0121766',
  'tt0121765',
  'tt0120915',
  'tt0120903',
  'tt0120815',
  'tt0120737',
  'tt0120735',
  'tt0120689']]

In [26]:
refined_dataset

,titleID,userID,primaryTitle,rating
0,tt0034583,ur0011104,Casablanca,1.0
1,tt0034583,ur0011762,Casablanca,10.0
2,tt0034583,ur0012295,Casablanca,7.0
3,tt0034583,ur0013691,Casablanca,10.0
4,tt0034583,ur0016344,Casablanca,10.0
...,...,...,...,...
103775,tt8946378,ur99708365,Knives Out,7.0
103776,tt8946378,ur99877336,Knives Out,8.0
103777,tt8946378,ur99907711,Knives Out,5.0
103778,tt8946378,641c87d06a97e629837fc079,Knives Out,8.0


In [27]:
movieUser_df

titleID,tt0034583,tt0050083,tt0054215,tt0060196,tt0062622,tt0066921,tt0068646,tt0071562,tt0071853,tt0073195,...,tt1205489,tt1211837,tt1219289,tt1228705,tt3748528,tt3896198,tt7131622,tt7286456,tt8579674,tt8946378
userID,,,,,,,,,,,,,,,,,,,,,
641c87d06a97e629837fc079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,0.0,8.0
645ad00ea917e05432316a5c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
ur0000066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ur99931473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
ur99955002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur99964320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
movieUser_df.loc[movieUser_df.index.values == "641c87d06a97e629837fc079"]

titleID,tt0034583,tt0050083,tt0054215,tt0060196,tt0062622,tt0066921,tt0068646,tt0071562,tt0071853,tt0073195,...,tt1205489,tt1211837,tt1219289,tt1228705,tt3748528,tt3896198,tt7131622,tt7286456,tt8579674,tt8946378
userID,,,,,,,,,,,,,,,,,,,,,
641c87d06a97e629837fc079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,0.0,8.0


In [29]:
movieUser_df.values[movieUser_df.index == "641c87d06a97e629837fc079"]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 3., 7., 0., 0., 0., 8.]])

In [30]:
target_user_id

'645ad00ea917e05432316a5c'

In [31]:
movieUser_df.index.values[0]

'641c87d06a97e629837fc079'

In [32]:
movieUser_df.index == "641c87d06a97e629837fc079"

array([ True, False, False, ..., False, False, False])

In [33]:
np.where(movieUser_df.index == "641c87d06a97e629837fc079")[0][0]

0

In [34]:
#movieUser_df = movieUser_df.reset_index(); 
movieUser_df[movieUser_df.userID=='ur99966337'].index

AttributeError: 'DataFrame' object has no attribute 'userID'

In [ ]:
movieUser_df

In [ ]:
movieUser_df[movieUser_df.userID=='ur99966337'].index[0]

In [ ]:
np.where(movieUser_df.index == target_user_id)[0][0]

In [57]:
np.asarray([movieUser_df.values[1]])

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,
         0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.